In [1]:
# check langchain version if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.29


In [2]:


# Initialize client
import os
import json
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from textwrap import indent
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
# Configure llm via .env

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Define a dynamic routing ruleset


In [3]:
ROUTING_RULES = """
1️⃣ If CUSTOMER_TIER = "Enterprise" AND SENTIMENT_SCORE < 0.3:
    Priority = "URGENT"
    Route to = "Senior Support"
    SLA_TARGET = "1 hour"
    ESCALATION_CRITERIA = "If not acknowledged in 30 minutes"

2️⃣ If PRODUCT_AREA = "Billing" AND EXTRACTED_KEYWORDS contain any of ["refund", "charge", "payment"]:
    Route to = "Billing Team"
    SLA_TARGET = "4 hours"
    TEMPLATE_ID = "Billing_Issue_Response"

3️⃣ If EXTRACTED_KEYWORDS contain any of ["bug", "error", "broken"] AND CUSTOMER_TIER = "Premium":
    Priority = "HIGH"
    Route to = "Technical Team"
    SLA_TARGET = "2 hours"
    TEMPLATE_ID = "Tech_Error_Response"

4️⃣ Default rule:
    Priority = "MEDIUM"
    Route to = "General Support"
    SLA_TARGET = "6 hours"
    TEMPLATE_ID = "Generic_Response"
    
    """

In [4]:
# Example input values — you can replace these dynamically
ticket_input = {
    "CUSTOMER_TIER": "Enterprise",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.25,
    "EXTRACTED_KEYWORDS": ["refund", "delay"]
}

In [5]:
# Build the dynamic prompt
prompt = f"""
You are an AI assistant for a Customer Support Ticket Analysis System.
Your task is to analyze the ticket details and determine routing priority,
SLA, escalation, and response template based on the rules.

[INPUTS]
Customer Tier: {ticket_input['CUSTOMER_TIER']}
Product Area: {ticket_input['PRODUCT_AREA']}
Sentiment Score: {ticket_input['SENTIMENT_SCORE']}
Extracted Keywords: {ticket_input['EXTRACTED_KEYWORDS']}

[ROUTING RULES]
{ROUTING_RULES}

[OUTPUT FORMAT]
Priority: <CALCULATED_PRIORITY>
Route to: <ASSIGNED_TEAM>
Suggested response time: <SLA_TARGET>
Initial response template: <TEMPLATE_ID>
Escalation trigger: <ESCALATION_CRITERIA>

Now output only the structured result, no explanation.
"""

In [7]:
#call the model



In [6]:
# --- TUNED PROMPT RESPONSE ---
dynamic_messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=prompt)
]
dynamic_response = llm(dynamic_messages).content.strip()

In [9]:
print("Dynamic PROMPT:")
print(indent(dynamic_response, "    "))

Dynamic PROMPT:
    Priority: URGENT  
    Route to: Senior Support  
    Suggested response time: 1 hour  
    Initial response template: Billing_Issue_Response  
    Escalation trigger: If not acknowledged in 30 minutes


# To Do: Sample ticket_input test cases to try One by one test and observe output


In [ ]:
#Test Case 1 — Enterprise + Low Sentiment (Rule 1)
ticket_input = {
    "CUSTOMER_TIER": "Enterprise",
    "PRODUCT_AREA": "App",
    "SENTIMENT_SCORE": 0.20,
    "EXTRACTED_KEYWORDS": ["slow", "crash"]
}
#Test Case 2 — Billing + Refund Keyword (Rule 2)
ticket_input = {
    "CUSTOMER_TIER": "Standard",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.75,
    "EXTRACTED_KEYWORDS": ["refund"]
}
#Test Case 3 — Billing but No Keyword (Default Rule)
ticket_input = {
    "CUSTOMER_TIER": "Standard",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.60,
    "EXTRACTED_KEYWORDS": ["question", "invoice"]
}
#Test Case 4 — Premium + Bug Keyword (Rule 3)
ticket_input = {
    "CUSTOMER_TIER": "Premium",
    "PRODUCT_AREA": "App",
    "SENTIMENT_SCORE": 0.55,
    "EXTRACTED_KEYWORDS": ["bug", "login failure"]
}
#Test Case 5 — Premium + No Bug Keyword (Default Rule)
ticket_input = {
    "CUSTOMER_TIER": "Enterprise",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.85,
    "EXTRACTED_KEYWORDS": ["question"]
}
#Test Case 6 — Enterprise but Good Sentiment (Default Rule)
ticket_input = {
    "CUSTOMER_TIER": "Enterprise",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.85,
    "EXTRACTED_KEYWORDS": ["question"]
}
#Test Case 7 — Multi-Trigger (Billing + Refund) — Should Use Rule 2
ticket_input = {
    "CUSTOMER_TIER": "Premium",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.40,
    "EXTRACTED_KEYWORDS": ["refund", "bug"]
}
#Test Case 9 — Enterprise + Bug + High Sentiment — Rule 3 Does NOT apply because tier ≠ Premium → Default Rule
ticket_input = {
    "CUSTOMER_TIER": "Enterprise",
    "PRODUCT_AREA": "App",
    "SENTIMENT_SCORE": 0.90,
    "EXTRACTED_KEYWORDS": ["bug"]
}
#Test Case 10 — Billing + Charge Keyword (Rule 2)
ticket_input = {
    "CUSTOMER_TIER": "Standard",
    "PRODUCT_AREA": "Billing",
    "SENTIMENT_SCORE": 0.33,
    "EXTRACTED_KEYWORDS": ["charge", "payment"]
}
